In [0]:
# Load the dataset from DBFS (Databricks Filesystem)
file_path = "/databricks-datasets/samples/population-vs-price/data_geo.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- 2014 rank: integer (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- State Code: string (nullable = true)
 |-- 2014 Population estimate: integer (nullable = true)
 |-- 2015 median sales price: double (nullable = true)

+---------+-------------+-------+----------+------------------------+-----------------------+
|2014 rank|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+---------+-------------+-------+----------+------------------------+-----------------------+
|      101|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|      125|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|      122|       Mobile|Alabama|        AL|                  194675|                  122.5|
|      114|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|       64|Anchorage[19]| Alaska|        AK|                  30101

In [0]:
# Select relevant columns for analysis
df_selected = df.select(
    "City", "State", "State Code", 
    "2014 Population estimate", "2015 median sales price"
)
df_selected.show(5)

+-------------+-------+----------+------------------------+-----------------------+
|         City|  State|State Code|2014 Population estimate|2015 median sales price|
+-------------+-------+----------+------------------------+-----------------------+
|   Birmingham|Alabama|        AL|                  212247|                  162.9|
|   Huntsville|Alabama|        AL|                  188226|                  157.7|
|       Mobile|Alabama|        AL|                  194675|                  122.5|
|   Montgomery|Alabama|        AL|                  200481|                  129.0|
|Anchorage[19]| Alaska|        AK|                  301010|                   null|
+-------------+-------+----------+------------------------+-----------------------+
only showing top 5 rows



In [0]:
#top 10 most populated cities
from pyspark.sql.functions import desc
populated_cities=df.select("City").orderBy(desc("2014 population estimate")).limit(10)
display(populated_cities)

City
New York[6]
Los Angeles
Chicago
Houston[7]
Philadelphia[8]
Phoenix
San Antonio
San Diego
Dallas
San Jose


Databricks visualization. Run in Databricks to view.

In [0]:
#Average median sales price by State
from pyspark.sql.functions import avg
# Group by State and calculate average of "2015 median sales price"
avg_price_by_state = df_selected.groupBy("State").agg(
    avg("2015 median sales price").alias("Average_Median_Sales_Price")
)

# Show the result
avg_price_by_state.orderBy("Average_Median_Sales_Price", ascending=False).show()



+--------------------+--------------------------+
|               State|Average_Median_Sales_Price|
+--------------------+--------------------------+
|             Hawai'i|                     699.3|
|          California|         547.9714285714286|
|District of Columbia|                     367.8|
|          New Jersey|                     350.8|
|            Colorado|        333.46666666666664|
|          Washington|        266.29999999999995|
|       Massachusetts|                     261.8|
|         Connecticut|        250.13333333333333|
|                Utah|                     243.3|
|       New Hampshire|                     237.4|
|              Nevada|                     237.3|
|        Rhode Island|                     233.3|
|              Oregon|        231.53333333333333|
|            Maryland|                     223.1|
|           Wisconsin|                     210.8|
|           Minnesota|                     209.4|
|            Virginia|                    201.25|


In [0]:
#Filter cities with population >1 million
from pyspark.sql.functions import desc
df_selected.filter(df_selected["2014 Population estimate"] > 1000000).select("City", "2014 Population estimate").orderBy(desc("2014 Population estimate")).show()



+---------------+------------------------+
|           City|2014 Population estimate|
+---------------+------------------------+
|    New York[6]|                 8491079|
|    Los Angeles|                 3928864|
|        Chicago|                 2722389|
|     Houston[7]|                 2239558|
|Philadelphia[8]|                 1560297|
|        Phoenix|                 1537058|
|    San Antonio|                 1436697|
|      San Diego|                 1381069|
|         Dallas|                 1281047|
|       San Jose|                 1015785|
+---------------+------------------------+



In [0]:
#Identify cities with missing or zero price values
df_selected.filter((df_selected["2015 median sales price"].isNull()) | (df_selected["2015 median sales price"] == 0)).show()


+-------------+----------+----------+------------------------+-----------------------+
|         City|     State|State Code|2014 Population estimate|2015 median sales price|
+-------------+----------+----------+------------------------+-----------------------+
|Anchorage[19]|    Alaska|        AK|                  301010|                   null|
|     Chandler|   Arizona|        AZ|                  254276|                   null|
|  Gilbert[20]|   Arizona|        AZ|                  239277|                   null|
|     Glendale|   Arizona|        AZ|                  237517|                   null|
|         Mesa|   Arizona|        AZ|                  464704|                   null|
|       Peoria|   Arizona|        AZ|                  166934|                   null|
|   Scottsdale|   Arizona|        AZ|                  230512|                   null|
|     Surprise|   Arizona|        AZ|                  126275|                   null|
|        Tempe|   Arizona|        AZ|      